In [1]:
#!/usr/bin/env python

import sys
import cv2 as cv
import numpy as np
from scipy.stats import norm
from KDEpy import NaiveKDE, TreeKDE, FFTKDE
from pynhhd import nHHD
import matplotlib.pyplot as plt

In [21]:
# 图像校正
def img_rectify(img): 
    mapx = np.load('./rectify para/mapx.npy')
    mapy = np.load('./rectify para/mapy.npy')
    image_remap = cv.remap(img, mapx, mapy, interpolation=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT)
    return image_remap

# 高斯滤波
def Gauss2_filter(img):
    img1 = cv.GaussianBlur(img,(7,7),0)	##参数1为输入图像，参数2为滤波器窗口大小，必须为正奇数，参数3为标准差
    img2 = cv.GaussianBlur(img1,(7,7),0)
    return img2

# 图像拓扑
def morph(img):
    kernel1 = np.ones((6,6), dtype=np.uint8)

    close = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel1, 6)

    open = cv.morphologyEx(close, cv.MORPH_OPEN, kernel1, 5)

    return open

# 图像锐化
def sharpen(img, sharpness=100, ktype=1):
  n = sharpness/100
  if ktype == 1:
    sharpen_op = np.array([[0 ,  -n,   0 ], \
                           [-n, 4*n+1, -n], \
                           [0 ,  -n,   0 ]], dtype=np.float32)
  if ktype == 2:
    sharpen_op = np.array([[-n,  -n,   -n], \
                           [-n, 8*n+1, -n], \
                           [-n,  -n,   -n]], dtype=np.float32)
  img_sharpen = cv.filter2D(img, cv.CV_32F, sharpen_op)
  img_sharpen = cv.convertScaleAbs(img_sharpen)
  return img_sharpen

# 图像预处理
def img_pre(img):
    img_croped = img_crop(img)
    #img_gauss = Gauss2_filter(img_croped)
    imggray = cv.cvtColor(img_croped, cv.COLOR_BGR2GRAY)
    #ret2, img_thresh =  cv.threshold(imggray, 75, 255, 1) 
    #img_morph = morph(img_thresh)
    #img_sharpen = sharpen(img_morph)

    return imggray

# 光流绘制
def draw_flow(img, flow, step=16):
    # 在间隔分开的像素采样点处绘制光流
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    # 创建线的终点
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines)
    # 创建图像并绘制
    vis = img #cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
    for (x1, y1), (x2, y2) in lines:
      cv.line(vis, (x1, y1), (x2, y2), (0, 0, 255), 1)
      cv.circle(vis, (x1, y1), 1, (0, 0, 255), -1)
    return vis

# 图像切割
def img_crop(img):
    crop = img[:, 80:560]
    return crop

# 图像逆映射
def image_warp(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv.remap(img, flow, None, cv.INTER_LINEAR)
    return res


# 图像相似度比较
def compare(img_ref, img_warped):
    img_ref_gray = cv.cvtColor(img_ref,cv.COLOR_BGR2GRAY)
    img_warped_gray = cv.cvtColor(img_warped,cv.COLOR_BGR2GRAY)
    D =  img_ref_gray - img_warped_gray
    temp = D > 200
    D[temp] = 256 - D[temp]
    mean = D.mean()
    return mean

# 多图展示
def show_multi_imgs(scale, imglist, order=None, border=5, border_color=(255, 255, 0)):
    """
    :param scale: float 原图缩放的尺度
    :param imglist: list 待显示的图像序列
    :param order: list or tuple 显示顺序 行×列
    :param border: int 图像间隔距离
    :param border_color: tuple 间隔区域颜色
    :return: 返回拼接好的numpy数组
    """
    if order is None:
        order = [1, len(imglist)]
    allimgs = imglist.copy()
    ws , hs = [], []
    for i, img in enumerate(allimgs):
        if np.ndim(img) == 2:
            allimgs[i] = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        allimgs[i] = cv.resize(img, dsize=(0, 0), fx=scale, fy=scale)
        ws.append(allimgs[i].shape[1])
        hs.append(allimgs[i].shape[0])
    w = max(ws)
    h = max(hs)
    # 将待显示图片拼接起来
    sub = int(order[0] * order[1] - len(imglist))
    # 判断输入的显示格式与待显示图像数量的大小关系
    if sub > 0:
        for s in range(sub):
            allimgs.append(np.zeros_like(allimgs[0]))
    elif sub < 0:
        allimgs = allimgs[:sub]
    imgblank = np.zeros(((h+border) * order[0], (w+border) * order[1], 3)) + border_color
    imgblank = imgblank.astype(np.uint8)
    for i in range(order[0]):
        for j in range(order[1]):
            imgblank[(i * h + i*border):((i + 1) * h+i*border), (j * w + j*border):((j + 1) * w + j*border), :] = allimgs[i * order[1] + j]
    return imgblank

# 光流轮廓
def outline(frame, flow,type=1):
    hsv = np.zeros_like(frame)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    if type == 1:
        hsv[...,1] = 255
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    elif type == 2:
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 1] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
        hsv[..., 2] = 255
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    return bgr

def create_pic_white(flow):
   dims = flow.shape
   img = np.ones((dims[0], dims[1]))*255
   return img

# Helmoltz decompose
def Helmholtz(flow):
    dims = flow.shape
    dims = dims[0:2]
    dx = (0.01, 0.01) 
    Y, X = np.mgrid[0:dims[0], 0:dims[1]]

    mvf = np.linalg.norm(flow, axis=2)

    #vrng = (0, 7.07106781187)
    vrng = (0, mvf.max())

    nhhd = nHHD(grid=dims, spacings=dx)
    nhhd.decompose(flow)

    r = nhhd.r
    d = nhhd.d
    h = nhhd.h

    Sn = np.linalg.norm(d, ord=1, axis=2)
    Sn = Sn.sum()/(dims[0]*dims[1])

    Sr = flow.sum(axis=0).sum(axis=0)
    St = ((Sr[0]/dims[0])**2+(Sr[1]/dims[1])**2)**(1/2)

    return Sn,St,d

def write_data(L, file):
    file = open("./data/"+file, mode='w+')
    for i  in range(len(L)):
        file.write(str(L[i])+"\n")
    file.close()


# 调用摄像头
def catch_video(name='opt_flow', index = 0):
    cap = cv.VideoCapture(index)
    dis = cv.DISOpticalFlow_create(2)
    if not cap.isOpened():
        # 如果没有检测到摄像头，报错
        raise Exception('Check if the camera is on.')
    if cap.isOpened():
        ret1, old_frame = cap.read()
        old_prev= img_pre(old_frame)
        rawdata_Sn = []
        rawdata_St = []
        cv.imwrite("./DIS/old_frame.jpg", old_frame)
        cv.imwrite("./DIS/old_prev.jpg", old_prev)
    while cap.isOpened(): 
        ret2, new_frame = cap.read()
        new_prev = img_pre(new_frame)
        flow  = np.zeros((len(new_frame[0]),len(new_frame[1]),2))
        flow = dis.calc(old_prev, new_prev, None)
        """
        Sn, St, d = Helmholtz(flow)
        #rawdata_Sn.append(Sn)
        #rawdata_St.append(St)

        Fn = -0.714368+6.911549*Sn
        if Fn<0:
            Fn = 0
        print(Fn)
        """

        lunkuo = outline(img_crop(old_frame), flow)
        
        img_warped = image_warp(new_frame, flow)
        """
        error = compare(old_frame, img_warped)
        if error > 5:
            old_frame = new_frame
            temp = flow
            flow = temp + flow
        """
        opt_flow = draw_flow(img_crop(new_frame), flow)
        results = show_multi_imgs(1, [opt_flow, lunkuo, img_warped],(2,2))
        cv.imshow(name, results)
        key = cv.waitKey(10)
        if key & 0xFF == ord('q'):
            # 按q退出
            break
        if cv.getWindowProperty('opt_flow', cv.WND_PROP_AUTOSIZE) < 1:
            # 点x退出
            break
    cap.release()
    cv.destroyAllWindows()
    #return rawdata_Sn, rawdata_St


In [22]:
if __name__ == "__main__":    
    #rawdata_Sn,rawdata_St = catch_video()
    catch_video()
    #write_data(rawdata_Sn, file="Sn6.txt")
    #write_data(rawdata_St, file="St6.txt")
